# Demonstrating the padding oracle attack 

In this notebook, we will study the padding oracle attack on CBC mode encryption. The code in the code boxes below allow you to play around with a padding oracle and observe its responses to different queries. 

Let's start by creating some encrypted block, which will serve as a challenge for us to break! Our goal will be to figure out the plaintext block behind this ciphertext block with the help of a padding oracle.

In [ ]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

key = get_random_bytes(16)  # we create a 16-byte random key
plaintext = get_random_bytes(16)  # we create a 16-byte random plaintext block
cipher = AES.new(key, AES.MODE_ECB)  # we encrypt the plaintext block with the key
ciphertext = cipher.encrypt(plaintext)  # to get the ciphertext block that the attacker has to break
print(ciphertext.hex())

1b957c4940d09b5468226d0158640d0c


OK, the above printed ciphertext block is our challenge: we have to figure out the plaintext block from which it was created. We have access to a padding oracle, which is defined in the code box below:

In [ ]:
from Crypto.Util import Padding

def oracle(x):
    global key   # the oracle knows the key that was used to create the challenge ciphertext
    cipher = AES.new(key, AES.MODE_CBC, x[:16])  # the first block of the input is interpreted as the IV
    paddded_plaintext = cipher.decrypt(x[16:])  # the rest of the input is the ciphertext that the oracle decrypts in CBC mode
    try:
        Padding.unpad(paddded_plaintext, 16, style='iso7816')   # the oracle tries unpadding the result
        return 'Padding OK'   # and return 'Padding OK' if the unpadding was successful
    except:
        return 'Padding error'  # otherwise the oracle returns 'Padding error' 

The attack starts here. We will send two-block inputs to the oracle where the first block `R` is a block that we constantly manipulate and the second block `Y` is the challenge ciphertext block that we want to break.

In [ ]:
Y = ciphertext

for r in range(256):
    R = bytes.fromhex('000000000000000000000000000000') + r.to_bytes(1)   # R consists of 00 bytes and only its last byte takes different values
    print(R.hex(), end='\r')
    if oracle(R+Y) == 'Padding OK': break   # when the padding is correct, we stop

So we found a block `R` (see above) such that for input `R`|`Y` we get a 'Padding OK' response from the oracle. We don't know, however, what specific padding was encountered by the oracle. The code below let you check which bytes of `R` affect the correctness of the padding, which ultimately let you figure out the length of the actual padding and hence the specific padding encountered. For this, just modify the bytes of `R`, run the code, and observe whether the padding remains correct or you get an error.

In [46]:
print(oracle(bytes.fromhex('0000000000000000000000000000005c')+Y))

Padding OK


At this point, we can most likely determine the last byte of the unknown plaintext...

In [ ]:
b = 0x5c ^ 0x80   # the last byte b of the plaintext can be computed like this
print(b.to_bytes(1).hex())
print(b == plaintext[-1])

dc
True


The code boxes below allow you to continue the attack along the same lines as above, and determine more bytes from the plaintext...

In [55]:
for r in range(256):
    R = bytes.fromhex('0000000000000000000000000000') + r.to_bytes(1) + bytes.fromhex('dc')
    print(R.hex(), end='\r')
    if oracle(R+Y) == 'Padding OK': break

In [56]:
print(oracle(bytes.fromhex('0000000000000000000000000000d7dc')+Y))

Padding OK


In [57]:
b = 0xd7 ^ 0x80
print(b.to_bytes(1).hex())
print(b == plaintext[-2])

57
True
